In [7]:
import pandas as pd
import re
import os

# 프로젝트 폴더 구조에 맞춘 경로 설정
RAW_DATA_PATH = "../data/01_raw/reviews_raw.csv"
PROCESSED_DATA_PATH = "../data/02_processed/"

# 폴더가 없으면 생성
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print("라이브러리 임포트 및 경로 설정 완료.")

라이브러리 임포트 및 경로 설정 완료.


In [8]:
# 1. 데이터 불러오기
try:
    df = pd.read_csv(RAW_DATA_PATH)
    print(f"데이터 로드 성공: 총 {len(df)}건")
    display(df.head(3)) # 상위 3개 확인
except FileNotFoundError:
    print(f"오류: {RAW_DATA_PATH} 파일이 없습니다. 크롤링 파일을 해당 위치로 옮겨주세요.")

데이터 로드 성공: 총 92592건


,Unnamed: 0,product_name,rating,skin_type,review
0,0,제품이름,별점,피부타입,리뷰
1,1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...
2,2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,"👉🏻 간단정리! - good! : 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨..."


In [9]:
def preprocess_text(text):
    """
    리뷰 텍스트 전처리 함수
    1. 결측치(NaN) 처리
    2. 한글, 영문, 숫자, 기본 문장부호(.,!?)를 제외한 특수문자(이모티콘 등) 제거
    3. 중복 공백 제거
    """
    if not isinstance(text, str):
        return ""

    # 1. 특수문자 제거 (한글, 영어, 숫자, 마침표, 쉼표, 느낌표, 물음표, 공백만 남김)
    # 이모티콘 등이 여기서 제거됩니다.
    text = re.sub(r"[^가-힣a-zA-Z0-9.,!?\s]", "", text)

    # 2. 줄바꿈 문자를 공백으로 변경
    text = text.replace("\n", " ")

    # 3. 중복 공백을 하나의 공백으로 줄임
    text = re.sub(r"\s+", " ", text).strip()

    return text

# 테스트
sample_text = "배송이 너무 빨라요!!! ㅎㅎㅎ 👍👍 100점 드립니다. ㅠㅠ"
print(f"전처리 전: {sample_text}")
print(f"전처리 후: {preprocess_text(sample_text)}")

전처리 전: 배송이 너무 빨라요!!! ㅎㅎㅎ 👍👍 100점 드립니다. ㅠㅠ
전처리 후: 배송이 너무 빨라요!!! 100점 드립니다.


In [10]:
# 1. 전처리 함수 적용
print("텍스트 전처리 진행 중...")
df['cleaned_review'] = df['review'].apply(preprocess_text)

# 2. 빈 리뷰 제거 (전처리 후 내용이 없는 경우)
df = df[df['cleaned_review'] != ""]

# 3. 너무 짧은 리뷰 제거 (예: 5글자 미만은 분석 가치가 낮음)
# "좋아요", "굿굿" 같은 단순 리뷰 제거 목적
df = df[df['cleaned_review'].str.len() >= 10]

# 4. 중복 리뷰 제거 (광고성 도배글 등 방지)
# cleaned_review가 완전히 똑같은 내용 제거
before_dedup = len(df)
df = df.drop_duplicates(subset=['cleaned_review'])
after_dedup = len(df)

print(f"-"*30)
print(f"중복 제거 전: {before_dedup}건")
print(f"중복 제거 후: {after_dedup}건")
print(f"삭제된 리뷰 수: {before_dedup - after_dedup}건")
print(f"-"*30)

# 5. 인덱스 재설정 (깔끔하게 0부터 다시 시작)
df = df.reset_index(drop=True)
df['id'] = df.index # 고유 ID 부여

# 결과 확인
print(f"최종 데이터 개수: {len(df)}건")
df[['product_name', 'review', 'cleaned_review']].head()

텍스트 전처리 진행 중...
------------------------------
중복 제거 전: 92571건
중복 제거 후: 92559건
삭제된 리뷰 수: 12건
------------------------------
최종 데이터 개수: 92559건


,product_name,review,cleaned_review
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...,Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"👉🏻 간단정리! - good! : 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨...","간단정리! good! 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨울엔 두겹 레..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),❤️나의 피부상태: 복합성+트러블 조금만 자극적이면 피부 뒤집어짐;💜장점: 여드름...,나의 피부상태 복합성트러블 조금만 자극적이면 피부 뒤집어짐장점 여드름 짜거나 엄청 ...
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),또 샀어요!!!! 진짜 닥터지 최고입니다.. 건성분들이 쓰시기에는 너무 빨리 날아가...,또 샀어요!!!! 진짜 닥터지 최고입니다.. 건성분들이 쓰시기에는 너무 빨리 날아가...
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"닥터지 크림은 워낙 유명하기도 하고,성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...","닥터지 크림은 워낙 유명하기도 하고,성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔..."


In [11]:
# 전체 정제된 데이터를 저장 (나중에 전체 예측용으로 사용)
save_path = os.path.join(PROCESSED_DATA_PATH, "reviews_cleaned.csv")
df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"전처리 완료 데이터 저장됨: {save_path}")

전처리 완료 데이터 저장됨: ../data/02_processed/reviews_cleaned.csv


In [12]:
# 1. 샘플링 개수 설정 (학습용 데이터)
SAMPLE_SIZE = 2500

# 2. 랜덤 샘플링 (random_state를 고정해야 나중에 다시 실행해도 같은 데이터가 뽑힘)
if len(df) > SAMPLE_SIZE:
    df_labeling = df.sample(n=SAMPLE_SIZE, random_state=42).copy()
else:
    df_labeling = df.copy()
    print("경고: 전체 데이터가 2,500건 미만입니다. 전체를 사용합니다.")

# 3. 라벨 컬럼 생성 (비워둠)
df_labeling['label'] = ""

# 4. 인덱스 재정렬 (0 ~ 2499)
df_labeling = df_labeling.reset_index(drop=True)
df_labeling['id'] = df_labeling.index # 라벨링용 새로운 ID 부여

# 5. 엑셀로 저장 (직접 눈으로 보거나 수정할 때 엑셀이 편함)
label_file_path = os.path.join(PROCESSED_DATA_PATH, "labeling_sample.xlsx")
df_labeling.to_excel(label_file_path, index=False)

print(f"라벨링용 샘플 파일 저장됨: {label_file_path}")
print(f"추출된 샘플 개수: {len(df_labeling)}건")

라벨링용 샘플 파일 저장됨: ../data/02_processed/labeling_sample.xlsx
추출된 샘플 개수: 2500건


In [13]:
def generate_llm_prompt(dataframe, start_idx, batch_size=50):
    """
    데이터프레임에서 특정 구간의 리뷰를 뽑아 LLM 프롬프트 형식으로 출력
    """
    end_idx = min(start_idx + batch_size, len(dataframe))
    subset = dataframe.iloc[start_idx:end_idx]

    print("="*20 + " 복사 시작 (Copy Start) " + "="*20)
    print(f"데이터 구간: {start_idx} ~ {end_idx-1} (총 {len(subset)}개)\n")

    # 프롬프트 텍스트 생성
    prompt_text = ""
    for idx, row in subset.iterrows():
        # ID와 리뷰 내용을 명확히 구분
        prompt_text += f"ID {row['id']}: {row['cleaned_review']}\n"

    print(prompt_text)
    print("="*20 + " 복사 끝 (Copy End) " + "="*20)

# ---------------------------------------------------------
# 사용법: start_index를 0, 50, 100... 으로 바꾸면서 실행하세요.
# ---------------------------------------------------------
START_INDEX = 2000  # 여기 숫자를 50씩 늘려가며 실행하세요 (0, 50, 100 ...)
BATCH_SIZE = 50  # 한 번에 질문할 개수

generate_llm_prompt(df_labeling, START_INDEX, BATCH_SIZE)

==================== 복사 시작 (Copy Start) ====================
데이터 구간: 700 ~ 749 (총 50개)

ID 700: 순하고 보습도 좋고 유통기한도 좋습니다.자주 구입하는 품목중 하나입니다
ID 701: 순해요. 정말 순해요. 뒤집어진 피부를 빠르게 진정시켜주고 원상태로 바꿔줘요. 향도 취향저격 입니당. 비싸고 양은 적지만 다음에도 또 구매할 예정입니다.
ID 702: 이름답게 비타민이라 그런가 뭔가 상큼하고 너머 무겁지 않아서 좋아용
ID 703: 닥터지 수분크림 항상 사용하는 제품입니다순하고 좋아요
ID 704: 흡수가 빠르고 진짜 자연스럽게 톤업도 되고 자외선 차단도 작지만 되서 좋습니당
ID 705: 피부가 민감하거나 뒤집어졌을 때 이거 지인짜 완전 듬뿍 바르고 자면은 다음날 많이 좋아져 있는 게 눈에 보여요!! 진짜 진정에 좋은 제품이에요!! 좀 연고처럼 제형이 꾸덕꾸덕해서 지성피부에는 좀 미끄덩 거릴 것 같아요 그래서 티존부위는 조금만 바르고 볼 주변에 듬뿍 올려줘요토너 하나 쓰고 이거 단독으로 바르면 끝!!
ID 706: 시카 세럼을 네통이상 잘 사용해서 이번에는겔 크림을 사봤는데 좋은것 같아요 다쓰면 또 살것 같아요
ID 707: 보타닉힐보였을 때 부터 완전 인기몰이 했던 판테놀 블레미쉬크림!! 이번에 바이오힐보로 리뉴얼 되면서 성분도 조금씩 업그레이드 됬다고해서 구매해봤어요!! 입소문으로 유명한 제품이지만 뭔가 선뜻 사고싶지는 않았는데 요즘 갑자기 트러블이 너무 올라오고 피지도 폭발이라 가벼운 시카크림 찾다가 구매결정!! 일단 제형은 엄청 묽은 젤 타입입니다! 많이 발라도 흡수도 빠르고 산뜻하게 마무리 되요!! 정말 지성분들에겐 좋을 듯한 크림! 하지만 저는 티존 빼고는 건성이라서 가을 겨울철에 쓰기에는 너무 가벼워서 마무리는 오일로 꾹꾹 눌러주면 완벽!! 트러블 진정은 조금 더 써봐야 알겠지만 사용감운 되게 좋았어요! 끈적임없고 향도 거의 무향입니다! 피부가 편안함을 느끼는 거 같기는해요!